<a href="https://colab.research.google.com/github/1000Roses/NLP_course/blob/main/summarize_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import 
import pandas as pd

In [45]:
import pandas as pd
import re
import numpy as np
import nltk, keras, string, re, html, math

from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from collections import Counter, defaultdict
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score, classification_report

In [32]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [9]:
# read file csv
df = pd.read_csv('/content/drive/MyDrive/spam.csv', encoding = "ISO-8859-1")

In [10]:
df

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [11]:
df = df.drop(columns=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'])

In [15]:
# normalize data
df["v1"]= df["v1"].str.lower() 
df["v2"]= df["v2"].str.lower() 

In [16]:
df.head()

,v1,v2
0,ham,"go until jurong point, crazy.. available only ..."
1,ham,ok lar... joking wif u oni...
2,spam,free entry in 2 a wkly comp to win fa cup fina...
3,ham,u dun say so early hor... u c already then say...
4,ham,"nah i don't think he goes to usf, he lives aro..."


In [20]:
def cleaning(data):
    clean = re.sub('<.*?>', ' ', str(data))            
#removes HTML tags
    clean = re.sub('\'.*?\s',' ', clean)               
#removes all hanging letters afer apostrophes (s in it's)
    clean = re.sub(r'http\S+',' ', clean)              
#removes URLs
    clean = re.sub('\W+',' ', clean)                   
#replacing the non alphanumeric characters
    return html.unescape(clean)
df['v2'] = df['v2'].apply(cleaning)

In [21]:
df.head()

,v1,v2
0,ham,go until jurong point crazy available only in ...
1,ham,ok lar joking wif u oni
2,spam,free entry in 2 a wkly comp to win fa cup fina...
3,ham,u dun say so early hor u c already then say
4,ham,nah i don think he goes to usf he lives around...


In [25]:
def tokenizing(data):
    review = data['v2']                            
    #tokenizing is done
    tokens = nltk.word_tokenize(review)
    return tokens
df['v2'] = df.apply(tokenizing, axis=1)

In [26]:
df.head()

,v1,v2
0,ham,"[go, until, jurong, point, crazy, available, o..."
1,ham,"[ok, lar, joking, wif, u, oni]"
2,spam,"[free, entry, in, 2, a, wkly, comp, to, win, f..."
3,ham,"[u, dun, say, so, early, hor, u, c, already, t..."
4,ham,"[nah, i, don, think, he, goes, to, usf, he, li..."


In [29]:
stop_words = set(stopwords.words('english'))
def remove_stops(data):
    my_list = data['v2']
    meaningful_words = [w for w in my_list if not w in stop_words]           #stopwords are removed from the tokenized data
    return (meaningful_words)
df['v2'] = df.apply(remove_stops, axis=1)

In [30]:
df.head()

,v1,v2
0,ham,"[go, jurong, point, crazy, available, bugis, n..."
1,ham,"[ok, lar, joking, wif, u, oni]"
2,spam,"[free, entry, 2, wkly, comp, win, fa, cup, fin..."
3,ham,"[u, dun, say, early, hor, u, c, already, say]"
4,ham,"[nah, think, goes, usf, lives, around, though]"


In [33]:
lemmatizer = WordNetLemmatizer()
def lemmatizing(data):
    my_list = data['v2']
    lemmatized_list = [lemmatizer.lemmatize(word) for word in my_list]    
  #lemmatizing is performed. It's more efficient than stemming.
    return (lemmatized_list)
df['v2'] = df.apply(lemmatizing, axis=1)

In [35]:
df.head()

,v1,v2
0,ham,"[go, jurong, point, crazy, available, bugis, n..."
1,ham,"[ok, lar, joking, wif, u, oni]"
2,spam,"[free, entry, 2, wkly, comp, win, fa, cup, fin..."
3,ham,"[u, dun, say, early, hor, u, c, already, say]"
4,ham,"[nah, think, go, usf, life, around, though]"


In [40]:
def rejoin_words(data):
    my_list = data['v2']
    joined_words = ( " ".join(my_list))                     
    #rejoins all stemmed words
    return joined_words
df['v2'] = df.apply(rejoin_words, axis=1)

In [41]:
df.head()

,v1,v2
0,ham,go jurong point crazy available bugis n great ...
1,ham,ok lar joking wif u oni
2,spam,free entry 2 wkly comp win fa cup final tkts 2...
3,ham,u dun say early hor u c already say
4,ham,nah think go usf life around though


In [37]:
encoder = LabelEncoder()
Y = encoder.fit_transform(df['v1'])

In [38]:
Y

array([0, 0, 1, ..., 0, 0, 0])

In [86]:
vectorizer = TfidfVectorizer( max_features = 10000 ,ngram_range = (1,3), binary = False)

X = vectorizer.fit_transform(df['v2'])

In [ ]:
# get feature from X
# vectorizer.get_feature_names()

In [87]:
X

<5572x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 64904 stored elements in Compressed Sparse Row format>

In [88]:
# split into train and test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
try:
  X_train = X_train.toarray()
except:
  pass

try:
  X_test = X_test.toarray()
except:
  pass

try:
  Y_train = Y_train.toarray()
except:
  pass

try:
  Y_test = Y_test.toarray()
except:
  pass


- Naive bayes

In [93]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
Y_gnb_pred = gnb.fit(X_train, Y_train).predict(X_test)

print("\nThe classification report with metrics - \n", classification_report(Y_test ,Y_gnb_pred))
print("Accuracy:", accuracy_score(Y_test, Y_gnb_pred))


The classification report with metrics - 
               precision    recall  f1-score   support

           0       0.99      0.92      0.95       965
           1       0.65      0.92      0.76       150

    accuracy                           0.92      1115
   macro avg       0.82      0.92      0.86      1115
weighted avg       0.94      0.92      0.93      1115

Accuracy: 0.9219730941704036


- SVM

In [94]:
#Import svm model
from sklearn import svm

#Create a svm Classifier
clf = svm.SVC(kernel='linear') # Linear Kernel

#Train the model using the training sets
clf.fit(X_train, Y_train)

#Predict the response for test dataset
Y_svm_pred = clf.predict(X_test)
print("\nThe classification report with metrics - \n", classification_report(Y_test ,Y_svm_pred))
print("Accuracy:", accuracy_score(Y_test, Y_svm_pred))


The classification report with metrics - 
               precision    recall  f1-score   support

           0       0.98      1.00      0.99       965
           1       0.97      0.86      0.91       150

    accuracy                           0.98      1115
   macro avg       0.97      0.93      0.95      1115
weighted avg       0.98      0.98      0.98      1115

Accuracy: 0.9775784753363229


- Perception

In [103]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(random_state=1, max_iter=600, early_stopping = True,  hidden_layer_sizes = (100,200,100,)).fit(X_train, Y_train)
Y_mlp_predict = clf.predict(X_test)
print("\nThe classification report with metrics - \n", classification_report(Y_test ,Y_mlp_predict))
print("Accuracy:", accuracy_score(Y_test, Y_mlp_predict))


The classification report with metrics - 
               precision    recall  f1-score   support

           0       0.98      0.99      0.99       965
           1       0.96      0.88      0.92       150

    accuracy                           0.98      1115
   macro avg       0.97      0.94      0.95      1115
weighted avg       0.98      0.98      0.98      1115

Accuracy: 0.97847533632287
